# Intent classifier

Description

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/intents/train_2.csv')
train

,Unnamed: 0,text,category,entity_type,entity_value,start_offset,end_offset,tags
0,0,how can I cancel purchase 113542617735902?,cancel_order,order_id,113542617735902,26.0,41.0,BIL
1,1,can you help me canceling purchase 00004587345?,cancel_order,order_id,00004587345,35.0,46.0,BIL
2,2,i want assistance to cancel purchase 732201349959,cancel_order,order_id,732201349959,37.0,49.0,BLQ
3,3,i want assistance to cancel order 732201349959,cancel_order,order_id,732201349959,34.0,46.0,BQ
4,4,"I don't want my last item, help me cancel orde...",cancel_order,order_id,370795561790,48.0,60.0,BCLN
...,...,...,...,...,...,...,...,...
6475,6475,I am waiting for a rebate of 299 dollars,track_refund,refund_amount,299,29.0,32.0,BL
6476,6476,I am waiting for a compensation of 1200 dollars,track_refund,refund_amount,1200,35.0,39.0,BL
6477,6477,I expect a compensation of 160 dollars,track_refund,refund_amount,160,27.0,30.0,BL
6478,6478,I am wating for a reimbursement of $350,track_refund,refund_amount,350,36.0,39.0,BLZ


In [3]:
test=pd.read_csv('/content/drive/MyDrive/intents/test_2.csv')

# EDA

In [4]:
train = train.rename(columns={"intent":"category", "utterance":"text"})

In [5]:
train['category'] = train['category'].replace({
    "cancel_order":"отменить_заказ",
    "change_order":"изменить_заказ",
    "change_shipping_address":"изменить_адрес_доставки",
    "check_cancellation_fee":"уточнить_стоимость_отмены",
    "check_invoice":"уточнить_по_чеку",
    "check_payment_methods":"уточнить_способ_оплаты",
    "check_refund_policy":"уточнить_политику_возврата",
    "create_account":"создать_аккаунт",
    "delete_account":"удалить_аккаунт",
    "delivery_options":"способы_доставки",
    "get_refund":"получить_возврат",
    "newsletter_subscription":"подписка_рассылку",
    "payment_issue":"проблема_при_оплате",
    "place_order":"разместить_заказ",
    "recover_password":"восстановить_пароль",
    "registration_problems":"проблема_при_регистрации",
    "review":"отзыв",
    "delivery_period":"время_доставки",
    "edit_account":"изменить аккаунт",
    "contact_human_agent":"обращение_к_оператору",
    "contact_customer_service":"обращение_к_службе_поддержки",
    "complaint":"пожаловаться",
    "set_up_shipping_address":"добавить_адрес_доставки",
    "switch_account":"сменить_аккаунт",
    "track_refund":"отследить_возврат",
    "get_invoice":"получить_чек",
    "track_order":"отследить_заказ"})

In [6]:
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.io as pio

value_counts = train.category.value_counts()
sorted_counts = value_counts.sort_values()


data = go.Bar(
    x=sorted_counts.values,
    y=sorted_counts.index,
    orientation='h',
    marker=dict(
        color='rgb(65,105,225)'
    )
)

layout = go.Layout(
    title='Распределение намерений',
    xaxis=dict(title='Число примеров'),
    yaxis=dict(title='Намерения'),
    height=800,
    width=800,
    margin=dict(l=100, r=50, t=50, b=50)
)

fig = go.Figure(data=[data], layout=layout)

pio.show(fig)


In [7]:
test = test.rename(columns={"intent":"category", "utterance":"text"})

Тестовый сет сделаем на русском

In [8]:
test['category'] = test['category'].replace({
    "cancel_order":"отменить_заказ",
    "change_order":"изменить_заказ",
    "change_shipping_address":"изменить_адрес_доставки",
    "check_cancellation_fee":"уточнить_стоимость_отмены",
    "check_invoice":"уточнить_по_чеку",
    "check_payment_methods":"уточнить_способ_оплаты",
    "check_refund_policy":"уточнить_политику_возврата",
    "create_account":"создать_аккаунт",
    "delete_account":"удалить_аккаунт",
    "delivery_options":"способы_доставки",
    "get_refund":"получить_возврат",
    "newsletter_subscription":"подписка_рассылку",
    "payment_issue":"проблема_при_оплате",
    "place_order":"разместить_заказ",
    "recover_password":"восстановить_пароль",
    "registration_problems":"проблема_при_регистрации",
    "review":"отзыв",
    "delivery_period":"время_доставки",
    "edit_account":"изменить аккаунт",
    "contact_human_agent":"обращение_к_оператору",
    "contact_customer_service":"обращение_к_службе_поддержки",
    "complaint":"пожаловаться",
    "set_up_shipping_address":"добавить_адрес_доставки",
    "switch_account":"сменить_аккаунт",
    "track_refund":"отследить_возврат",
    "get_invoice":"получить_чек",
    "track_order":"отследить_заказ"})

In [9]:
test['text'] = pd.read_csv('text_russian.csv')['текст']

In [10]:
test.to_csv('/content/drive/MyDrive/intents/test_2_russian.csv')

# LaBSE

In [ ]:
!pip install transformers sentence_transformers

In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_texts, test_texts, train_labels, test_labels = train_test_split(
    train['text'], train['category'], test_size=0.15, random_state=42)

encoder = SentenceTransformer('LaBSE')

train_embeddings = encoder.encode(train_texts.to_list()).astype(np.float32)
test_embeddings = encoder.encode(test_texts.to_list()).astype(np.float32)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels.tolist())
test_labels = label_encoder.transform(test_labels.tolist())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

# Simple network

In [14]:
model_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                49216     
                                                                 
 dense_1 (Dense)             (None, 27)                1755      
                                                                 
Total params: 50971 (199.11 KB)
Trainable params: 50971 (199.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)))
model_nn.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_nn.fit(train_embeddings, train_labels, validation_data=(test_embeddings, test_labels), epochs=30, batch_size=64)

loss, accuracy = model_nn.evaluate(test_embeddings, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/30
87/87 [==============================] - 6s 22ms/step - loss: 2.7508 - accuracy: 0.5610 - val_loss: 2.0098 - val_accuracy: 0.8560
Epoch 2/30
87/87 [==============================] - 1s 7ms/step - loss: 1.3365 - accuracy: 0.9118 - val_loss: 0.8189 - val_accuracy: 0.9424
Epoch 3/30
87/87 [==============================] - 1s 7ms/step - loss: 0.5638 - accuracy: 0.9579 - val_loss: 0.4089 - val_accuracy: 0.9794
Epoch 4/30
87/87 [==============================] - 1s 7ms/step - loss: 0.3102 - accuracy: 0.9775 - val_loss: 0.2559 - val_accuracy: 0.9856
Epoch 5/30
87/87 [==============================] - 1s 8ms/step - loss: 0.2031 - accuracy: 0.9822 - val_loss: 0.1787 - val_accuracy: 0.9907
Epoch 6/30
87/87 [==============================] - 1s 8ms/step - loss: 0.1449 - accuracy: 0.9878 - val_loss: 0.1346 - val_accuracy: 0.9918
Epoch 7/30
87/87 [==============================] - 1s 8ms/step - loss: 0.1103 - accuracy: 0.9902 - val_loss: 0.1059 - val_accuracy: 0.9918
Epoch 8/30
87/87 [=

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X_test, y_test = test['text'], test['category']

test_embeddings = encoder.encode(X_test.to_list()).astype(np.float32)
test_labels = label_encoder.transform(y_test.tolist())

yprob = model_nn.predict(test_embeddings)
yclasses=yprob.argmax(axis=-1)
y_pred = label_encoder.inverse_transform(yclasses)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print("Confusion Matrix:")

26/26 [==============================] - 0s 5ms/step
Accuracy: 0.9950617283950617
Precision: 0.9952388121764815
Recall: 0.9950617283950617
F1-score: 0.9950649904758323
Confusion Matrix:


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np
import pandas as pd
import seaborn as sns

class_labels = train['category'].unique()

tickvals = list(range(len(class_labels)))
ticktext = class_labels

heatmap = go.Heatmap(z=confusion,
                     x=tickvals,
                     y=tickvals,
                     x0=0.5,
                     y0=0.5,
                     colorscale='Blues',
                     zmin=0,
                     zmax=50,
                     colorbar=dict(title='Count'),
                     hoverongaps=False)

layout = go.Layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted', tickvals=tickvals, ticktext=ticktext),
    yaxis=dict(title='True', tickvals=tickvals, ticktext=ticktext, automargin=True),
    height=800
)

fig = go.Figure(data=[heatmap], layout=layout)

pyo.iplot(fig)


# mBERT

In [ ]:
!pip install transformers sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_texts, test_texts, train_labels, test_labels = train_test_split(
    train['text'], train['category'], test_size=0.15, random_state=42)

encoder = SentenceTransformer('bert-base-multilingual-cased')

train_embeddings = encoder.encode(train_texts.to_list()).astype(np.float32)
test_embeddings = encoder.encode(test_texts.to_list()).astype(np.float32)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels.tolist())
test_labels = label_encoder.transform(test_labels.tolist())


.gitattributes:   0%|          | 0.00/445 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

# Simple network

In [ ]:
model_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                49216     
                                                                 
 dense_1 (Dense)             (None, 27)                1755      
                                                                 
Total params: 50971 (199.11 KB)
Trainable params: 50971 (199.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)))
model_nn.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_nn.fit(train_embeddings, train_labels, validation_data=(test_embeddings, test_labels), epochs=30, batch_size=64)

loss, accuracy = model_nn.evaluate(test_embeddings, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/30
87/87 [==============================] - 2s 8ms/step - loss: 1.8795 - accuracy: 0.5957 - val_loss: 0.8456 - val_accuracy: 0.8652
Epoch 2/30
87/87 [==============================] - 0s 4ms/step - loss: 0.5287 - accuracy: 0.9201 - val_loss: 0.3911 - val_accuracy: 0.9342
Epoch 3/30
87/87 [==============================] - 1s 6ms/step - loss: 0.2807 - accuracy: 0.9539 - val_loss: 0.2594 - val_accuracy: 0.9609
Epoch 4/30
87/87 [==============================] - 1s 8ms/step - loss: 0.1851 - accuracy: 0.9706 - val_loss: 0.1935 - val_accuracy: 0.9681
Epoch 5/30
87/87 [==============================] - 1s 6ms/step - loss: 0.1366 - accuracy: 0.9786 - val_loss: 0.1545 - val_accuracy: 0.9743
Epoch 6/30
87/87 [==============================] - 1s 6ms/step - loss: 0.1006 - accuracy: 0.9858 - val_loss: 0.1305 - val_accuracy: 0.9763
Epoch 7/30
87/87 [==============================] - 0s 5ms/step - loss: 0.0789 - accuracy: 0.9913 - val_loss: 0.1118 - val_accuracy: 0.9794
Epoch 8/30
87/87 [==

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X_test, y_test = test['text'], test['category']

test_embeddings = encoder.encode(X_test.to_list()).astype(np.float32)
test_labels = label_encoder.transform(y_test.tolist())

yprob = model_nn.predict(test_embeddings)
yclasses=yprob.argmax(axis=-1)
y_pred = label_encoder.inverse_transform(yclasses)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print("Confusion Matrix:")

26/26 [==============================] - 0s 4ms/step
Accuracy: 0.4074074074074074
Precision: 0.557727590160117
Recall: 0.4074074074074074
F1-score: 0.3950860884874609
Confusion Matrix:


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np
import pandas as pd
import seaborn as sns

class_labels = train['category'].unique()

tickvals = list(range(len(class_labels)))
ticktext = class_labels

heatmap = go.Heatmap(z=confusion,
                     x=tickvals,
                     y=tickvals,
                     x0=0.5,
                     y0=0.5,
                     colorscale='Blues',
                     zmin=0,
                     zmax=50,
                     colorbar=dict(title='Count'),
                     hoverongaps=False)

layout = go.Layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted', tickvals=tickvals, ticktext=ticktext),
    yaxis=dict(title='True', tickvals=tickvals, ticktext=ticktext, automargin=True),
    height=800
)

fig = go.Figure(data=[heatmap], layout=layout)

pyo.iplot(fig)


# xlm roberta

In [ ]:
!pip install transformers sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_texts, test_texts, train_labels, test_labels = train_test_split(
    train['text'], train['category'], test_size=0.15, random_state=42)

encoder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

train_embeddings = encoder.encode(train_texts.to_list()).astype(np.float32)
test_embeddings = encoder.encode(test_texts.to_list()).astype(np.float32)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels.tolist())
test_labels = label_encoder.transform(test_labels.tolist())


.gitattributes:   0%|          | 0.00/795 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

# Simple network

In [ ]:
model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)))
model_nn.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_nn.fit(train_embeddings, train_labels, validation_data=(test_embeddings, test_labels), epochs=30, batch_size=64)

loss, accuracy = model_nn.evaluate(test_embeddings, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/30
87/87 [==============================] - 3s 9ms/step - loss: 0.9384 - accuracy: 0.7732 - val_loss: 0.2481 - val_accuracy: 0.9383
Epoch 2/30
87/87 [==============================] - 1s 8ms/step - loss: 0.1759 - accuracy: 0.9588 - val_loss: 0.1499 - val_accuracy: 0.9516
Epoch 3/30
87/87 [==============================] - 1s 8ms/step - loss: 0.1030 - accuracy: 0.9771 - val_loss: 0.1115 - val_accuracy: 0.9753
Epoch 4/30
87/87 [==============================] - 1s 9ms/step - loss: 0.0692 - accuracy: 0.9864 - val_loss: 0.0944 - val_accuracy: 0.9753
Epoch 5/30
87/87 [==============================] - 1s 8ms/step - loss: 0.0533 - accuracy: 0.9887 - val_loss: 0.0868 - val_accuracy: 0.9856
Epoch 6/30
87/87 [==============================] - 1s 9ms/step - loss: 0.0390 - accuracy: 0.9931 - val_loss: 0.0908 - val_accuracy: 0.9743
Epoch 7/30
87/87 [==============================] - 1s 8ms/step - loss: 0.0307 - accuracy: 0.9949 - val_loss: 0.0738 - val_accuracy: 0.9815
Epoch 8/30
87/87 [==

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X_test, y_test = test['text'], test['category']

test_embeddings = encoder.encode(X_test.to_list()).astype(np.float32)
test_labels = label_encoder.transform(y_test.tolist())

yprob = model_nn.predict(test_embeddings)
yclasses=yprob.argmax(axis=-1)
y_pred = label_encoder.inverse_transform(yclasses)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print("Confusion Matrix:")

26/26 [==============================] - 0s 2ms/step
Accuracy: 0.9617283950617284
Precision: 0.9659698543881392
Recall: 0.9617283950617284
F1-score: 0.9614027592927277
Confusion Matrix:


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np
import pandas as pd
import seaborn as sns

class_labels = train['category'].unique()

tickvals = list(range(len(class_labels)))
ticktext = class_labels

heatmap = go.Heatmap(z=confusion,
                     x=tickvals,
                     y=tickvals,
                     x0=0.5,
                     y0=0.5,
                     colorscale='Blues',
                     zmin=0,
                     zmax=50,
                     colorbar=dict(title='Count'),
                     hoverongaps=False)

layout = go.Layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted', tickvals=tickvals, ticktext=ticktext),
    yaxis=dict(title='True', tickvals=tickvals, ticktext=ticktext, automargin=True),
    height=800
)

fig = go.Figure(data=[heatmap], layout=layout)

pyo.iplot(fig)


# mT5

In [ ]:
!pip install transformers sentence_transformers

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_texts, test_texts, train_labels, test_labels = train_test_split(
    train['text'], train['category'], test_size=0.15, random_state=42)

encoder = SentenceTransformer('google/mt5-small')

train_embeddings = encoder.encode(train_texts.to_list(), show_progress_bar=True).astype(np.float32)
test_embeddings = encoder.encode(test_texts.to_list(), show_progress_bar=True).astype(np.float32)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels.tolist())
test_labels = label_encoder.transform(test_labels.tolist())


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning:

The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.



Batches:   0%|          | 0/173 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

# Simple network

In [18]:
model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 27)                1755      
                                                                 
Total params: 34587 (135.11 KB)
Trainable params: 34587 (135.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)))
model_nn.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_nn.fit(train_embeddings, train_labels, validation_data=(test_embeddings, test_labels), epochs=30, batch_size=64)

loss, accuracy = model_nn.evaluate(test_embeddings, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/30
87/87 [==============================] - 1s 6ms/step - loss: 2.8546 - accuracy: 0.3195 - val_loss: 2.2316 - val_accuracy: 0.5936
Epoch 2/30
87/87 [==============================] - 0s 5ms/step - loss: 1.7776 - accuracy: 0.6981 - val_loss: 1.3732 - val_accuracy: 0.7551
Epoch 3/30
87/87 [==============================] - 0s 4ms/step - loss: 1.1053 - accuracy: 0.8199 - val_loss: 0.8923 - val_accuracy: 0.8344
Epoch 4/30
87/87 [==============================] - 0s 4ms/step - loss: 0.7497 - accuracy: 0.8725 - val_loss: 0.6511 - val_accuracy: 0.8745
Epoch 5/30
87/87 [==============================] - 0s 4ms/step - loss: 0.5601 - accuracy: 0.9060 - val_loss: 0.5121 - val_accuracy: 0.8981
Epoch 6/30
87/87 [==============================] - 0s 4ms/step - loss: 0.4387 - accuracy: 0.9319 - val_loss: 0.4263 - val_accuracy: 0.9167
Epoch 7/30
87/87 [==============================] - 0s 4ms/step - loss: 0.3610 - accuracy: 0.9430 - val_loss: 0.3600 - val_accuracy: 0.9290
Epoch 8/30
87/87 [==

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X_test, y_test = test['text'], test['category']

test_embeddings = encoder.encode(X_test.to_list()).astype(np.float32)
test_labels = label_encoder.transform(y_test.tolist())

yprob = model_nn.predict(test_embeddings)
yclasses=yprob.argmax(axis=-1)
y_pred = label_encoder.inverse_transform(yclasses)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print("Confusion Matrix:")

26/26 [==============================] - 0s 2ms/step
Accuracy: 0.5888888888888889
Precision: 0.7061908373552931
Recall: 0.5888888888888889
F1-score: 0.5857963842331106
Confusion Matrix:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [20]:
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np
import pandas as pd
import seaborn as sns

class_labels = train['category'].unique()

tickvals = list(range(len(class_labels)))
ticktext = class_labels

heatmap = go.Heatmap(z=confusion,
                     x=tickvals,
                     y=tickvals,
                     x0=0.5,
                     y0=0.5,
                     colorscale='Blues',
                     zmin=0,
                     zmax=50,
                     colorbar=dict(title='Count'),
                     hoverongaps=False)

layout = go.Layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted', tickvals=tickvals, ticktext=ticktext),
    yaxis=dict(title='True', tickvals=tickvals, ticktext=ticktext, automargin=True),
    height=800
)

fig = go.Figure(data=[heatmap], layout=layout)

pyo.iplot(fig)


# XLM (Cross-lingual Language Model)

In [ ]:
!pip install transformers sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_texts, test_texts, train_labels, test_labels = train_test_split(
    train['text'], train['category'], test_size=0.15, random_state=42)

encoder = SentenceTransformer('xlm-r-100langs-bert-base-nli-mean-tokens')

train_embeddings = encoder.encode(train_texts.to_list()).astype(np.float32)
test_embeddings = encoder.encode(test_texts.to_list()).astype(np.float32)

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels.tolist())
test_labels = label_encoder.transform(test_labels.tolist())


.gitattributes:   0%|          | 0.00/574 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

# Simple network

In [ ]:
model_nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 64)                65600     
                                                                 
 dense_14 (Dense)            (None, 27)                1755      
                                                                 
Total params: 67355 (263.11 KB)
Trainable params: 67355 (263.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_nn = Sequential()
model_nn.add(Dense(64, activation='relu', input_shape=(train_embeddings.shape[1],)))
model_nn.add(Dense(len(label_encoder.classes_), activation='softmax'))

model_nn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_nn.fit(train_embeddings, train_labels, validation_data=(test_embeddings, test_labels), epochs=30, batch_size=64)

loss, accuracy = model_nn.evaluate(test_embeddings, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

Epoch 1/30
87/87 [==============================] - 2s 9ms/step - loss: 1.4593 - accuracy: 0.6572 - val_loss: 0.4807 - val_accuracy: 0.9136
Epoch 2/30
87/87 [==============================] - 1s 8ms/step - loss: 0.3269 - accuracy: 0.9372 - val_loss: 0.2443 - val_accuracy: 0.9444
Epoch 3/30
87/87 [==============================] - 1s 6ms/step - loss: 0.1856 - accuracy: 0.9668 - val_loss: 0.1788 - val_accuracy: 0.9630
Epoch 4/30
87/87 [==============================] - 0s 4ms/step - loss: 0.1306 - accuracy: 0.9762 - val_loss: 0.1290 - val_accuracy: 0.9691
Epoch 5/30
87/87 [==============================] - 0s 3ms/step - loss: 0.0970 - accuracy: 0.9831 - val_loss: 0.1164 - val_accuracy: 0.9763
Epoch 6/30
87/87 [==============================] - 0s 5ms/step - loss: 0.0793 - accuracy: 0.9851 - val_loss: 0.0984 - val_accuracy: 0.9774
Epoch 7/30
87/87 [==============================] - 1s 9ms/step - loss: 0.0625 - accuracy: 0.9909 - val_loss: 0.0940 - val_accuracy: 0.9794
Epoch 8/30
87/87 [==

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

X_test, y_test = test['text'], test['category']

test_embeddings = encoder.encode(X_test.to_list()).astype(np.float32)
test_labels = label_encoder.transform(y_test.tolist())

yprob = model_nn.predict(test_embeddings)
yclasses=yprob.argmax(axis=-1)
y_pred = label_encoder.inverse_transform(yclasses)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
confusion = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print("Confusion Matrix:")

26/26 [==============================] - 0s 2ms/step
Accuracy: 0.9012345679012346
Precision: 0.9127569576662491
Recall: 0.9012345679012346
F1-score: 0.9003283385590358
Confusion Matrix:


In [ ]:
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np
import pandas as pd
import seaborn as sns

class_labels = train['category'].unique()

tickvals = list(range(len(class_labels)))
ticktext = class_labels

heatmap = go.Heatmap(z=confusion,
                     x=tickvals,
                     y=tickvals,
                     x0=0.5,
                     y0=0.5,
                     colorscale='Blues',
                     zmin=0,
                     zmax=50,
                     colorbar=dict(title='Count'),
                     hoverongaps=False)

layout = go.Layout(
    title='Confusion Matrix',
    xaxis=dict(title='Predicted', tickvals=tickvals, ticktext=ticktext),
    yaxis=dict(title='True', tickvals=tickvals, ticktext=ticktext, automargin=True),
    height=800
)

fig = go.Figure(data=[heatmap], layout=layout)

pyo.iplot(fig)
